# Convolutional Neural Network

### Importing the Libraries

In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

2024-07-31 12:32:46.036798: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-31 12:32:46.091702: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-31 12:32:46.108307: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-31 12:32:46.146256: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-31 12:32:49.188518: W tensorflow/compiler/tf2

#### Print the version of tensorflow

In [3]:
tf.__version__

NameError: name 'tf' is not defined

In [3]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))


Num GPUs Available:  0


2024-07-31 12:32:52.701884: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


#### Confirm that tensorflow is actually using a GPU

In [4]:
import tensorflow as tf

# Print TensorFlow version
print("TensorFlow version:", tf.__version__)

# List all available physical devices
physical_devices = tf.config.list_physical_devices()
print("Available devices:")
for device in physical_devices:
    print(device)

# List available GPUs
gpus = tf.config.list_physical_devices('GPU')
print(f"Number of GPUs available: {len(gpus)}")

if gpus:
    print("GPUs:")
    for gpu in gpus:
        print(gpu)
else:
    print("No GPUs available")


TensorFlow version: 2.17.0
Available devices:
PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')
Number of GPUs available: 0
No GPUs available


# Par 1 - Data Preprocessing 

## Preprocessing the Training Set
This is done by applying some **transformations** on the training set to avoid overfitting.

**What are the transformations?**
* Image augmentation

### The ImageDataGenerator class

The ImageDataGenerator class in Keras is a powerful tool for performing real-time data augmentation and preprocessing on images. It allows you to apply various transformations to your images such as rescaling, shearing, and zooming.

### Create an instance of the ImageDataGenerator class


In [5]:
train_datagen = ImageDataGenerator(
    rescale=1.0/255,          # Rescale pixel values from [0, 255] to [0, 1]
    shear_range=0.2,          # Shear intensity (shear angle in counter-clockwise direction as radians)
    zoom_range=0.2,           # Range for random zoom
    horizontal_flip=True      # Randomly flip inputs horizontally
)

### Explanation:
**rescale=1.0/255:** Rescales the pixel values of the images to the range [0, 1].

**shear_range=0.2:** Applies random shearing transformations to the images.

**zoom_range=0.2:** Applies random zoom transformations to the images.

**horizontal_flip=True:** Randomly flips the images horizontally.

**flow_from_directory:** Generates batches of augmented image data from the directory.

### Generate batches of augmented image data

In [6]:
# Directory path where the images are stored
image_directory = "dataset/training_set/"

training_set = train_datagen.flow_from_directory(
    image_directory,
    target_size=(150, 150),   # Resize images to 150x150 (This made the training time ver long)
                              # Try with 64x64
    batch_size=32,            # Number of images to yield per batch
    class_mode='binary'       # Type of label arrays (binary/multiclass) -- (cat/dog - binary will make sense)
)

Found 8000 images belonging to 2 classes.


## Preprocessing teh Test Set

In [7]:
# Create an instance of the ImageDataGenerator for test with only rescaling
test_datagen = ImageDataGenerator(
    rescale=1.0/255           # Rescale pixel values from [0, 255] to [0, 1]
)

# Directory path where the test images are stored
test_image_directory = "dataset/test_set/"

# Generate batches of test image data
test_set = test_datagen.flow_from_directory(
    test_image_directory,
    target_size=(150, 150),   # Resize images to 64x64
    batch_size=32,            # Number of images to yield per batch
    class_mode='binary',      # Type of label arrays (binary/multiclass)
    shuffle=False             # Do not shuffle the data for evaluation
)

Found 2000 images belonging to 2 classes.


# Part 2 - Building the Convolutional Neural Network (CNN)

#### Initializing teh CNN

In [8]:
from tensorflow.keras.models import Sequential

cnn = Sequential()

In [9]:
cnn

<Sequential name=sequential, built=False>

### Step 1 - Convolutional Layer

In [10]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Input

# Add an Input layer
cnn.add(Input(shape=(150, 150, 3)))

# First convolutional layer
cnn.add(Conv2D(filters=32, kernel_size=3, activation='relu'))

### Step 2 - Pooling Layer

In [11]:
cnn.add(MaxPooling2D(pool_size=(2,2), strides=2))

### Adding a second Convolutional Layer

In [12]:
# First convolutional layer
cnn.add(Conv2D(filters=32, kernel_size=3, activation='relu'))
cnn.add(MaxPooling2D(pool_size=2, strides=2))

### Step 3 - Flattening 

In [13]:
from tensorflow.keras.layers import Flatten

# Flatten the results to feed into a DNN
cnn.add(Flatten())

### Step 4 - Full Connection

In [14]:
from tensorflow.keras.layers import Dense

cnn.add(Dense(units=512, activation='relu')) # The hidden neuron can be, units=128

2024-07-31 12:32:54.022810: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 84934656 exceeds 10% of free system memory.
2024-07-31 12:32:54.158982: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 84934656 exceeds 10% of free system memory.
2024-07-31 12:32:54.192414: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 84934656 exceeds 10% of free system memory.


### Step 5 - Output Layer

In [15]:
from tensorflow.keras.layers import Dense

cnn.add(Dense(units=1, activation='sigmoid')) # Binary classification, for 'dog' and 'cat'

# Part 3 - Training the Convolutional Neural Network (CNN)

### Compiling the CNN

In [16]:
### For binary classifiation, loss = "binary_crossentropy"
### For Non-binary classification, loss = "categorical_crossentropy"

cnn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

### Training the CNN on the Training set and evaluating it on the Test set

In [17]:
# Train model
history = cnn.fit(
    x=training_set,
    epochs=25,
    validation_data=test_set,
)

Epoch 1/25


2024-07-31 12:32:57.758952: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 84934656 exceeds 10% of free system memory.
/home/agbor/anaconda3/envs/tf_gpu/lib/python3.9/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
2024-07-31 12:33:02.914909: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 89718784 exceeds 10% of free system memory.


250/250 ━━━━━━━━━━━━━━━━━━━━ 338s 1s/step - accuracy: 0.5155 - loss: 1.1974 - val_accuracy: 0.5600 - val_loss: 0.6869
Epoch 2/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 361s 1s/step - accuracy: 0.5811 - loss: 0.6778 - val_accuracy: 0.6805 - val_loss: 0.6165
Epoch 3/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 315s 1s/step - accuracy: 0.6640 - loss: 0.6229 - val_accuracy: 0.6950 - val_loss: 0.6054
Epoch 4/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 312s 1s/step - accuracy: 0.6952 - loss: 0.5910 - val_accuracy: 0.7060 - val_loss: 0.5776
Epoch 5/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 324s 1s/step - accuracy: 0.7304 - loss: 0.5469 - val_accuracy: 0.7420 - val_loss: 0.5314
Epoch 6/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 316s 1s/step - accuracy: 0.7458 - loss: 0.5218 - val_accuracy: 0.7230 - val_loss: 0.5619
Epoch 7/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 308s 1s/step - accuracy: 0.7658 - loss: 0.4883 - val_accuracy: 0.7760 - val_loss: 0.4964
Epoch 8/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 317s 1s/step - accuracy: 0.7851 - loss: 0.4542 - val_accuracy: 0.768

# Part 4 - Making a single prediction

In [18]:
# import numpy as np
# from keras.preprocessing import image

# image_path = "dataset/single_prediction/cat_or_dog_1.png"
# test_image = image.load_img(image_path, target_size=(64, 64))

# ### Convert this test_image into an array
# test_image = image.img_to_array(test_image)

# ### Adding a batch_size to the image
# test_imagest_image = np.expand_dims(test_image, axis=0)

# ### Predicting the result
# result = cnn.predict(test_image)

# training_set.class_indices

# if result[0][0] == 1:
#     prediction = "dog"
# else:
#     prediction = "cat"

ValueError: Exception encountered when calling Sequential.call().

[1mInvalid input shape for input Tensor("data:0", shape=(32, 64, 3), dtype=float32). Expected shape (None, 150, 150, 3), but input has incompatible shape (32, 64, 3)[0m

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=(32, 64, 3), dtype=float32)
  • training=False
  • mask=None

In [ ]:
from keras.preprocessing import image
import numpy as np

# Load the image and resize it to 64x64
image_path = "dataset/single_prediction/cat_or_dog_6.png"

test_image = image.load_img(image_path, target_size=(150, 150))
test_image = image.img_to_array(test_image)

# Expand dimensions to add the batch size
test_image = np.expand_dims(test_image, axis=0)

# Predicting the result
result = cnn.predict(test_image)

training_set.class_indices

if result[0][0] == 1:
    prediction = "Class Dog"
else:
    prediction = "Class Cat"


In [ ]:
print(prediction)

### Evaluate the model on a dataset

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

def evaluate_model(model, test_generator):
    # Predict the results for the test set
    test_images, test_labels = next(test_generator)  # Or loop through your test data
    predictions = model.predict(test_images)
    predicted_classes = (predictions > 0.5).astype("int32")

    # Print the classification report
    print(classification_report(test_labels, predicted_classes))
    
    # Print confusion matrix
    print(confusion_matrix(test_labels, predicted_classes))



In [ ]:
loss, accuracy = cnn.evaluate(test_set, verbose=1)

print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy:.4f}")


### Evaluate Performance Metrics:

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

# Generate predictions
test_images, test_labels = next(test_set)  # Or loop through your test data
predictions = cnn.predict(test_images)
predicted_classes = (predictions > 0.5).astype("int32")

# Print classification report
print(classification_report(test_labels, predicted_classes, target_names=['Dog', 'Cat']))

# Print confusion matrix
print(confusion_matrix(test_labels, predicted_classes))


### Visualize Results:

In [ ]:
import matplotlib.pyplot as plt

def plot_predictions(image_paths, predictions):
    for img_path, pred in zip(image_paths, predictions):
        img = image.load_img(img_path, target_size=(150, 150))
        plt.imshow(img)
        plt.title(f"Prediction: {'Dog' if pred > 0.5 else 'Cat'}")
        plt.show()

# Assuming you have paths and predictions
test_images = "dataset/test_set/"
predictions = cnn.predict(test_images)
plot_predictions(test_image_paths, predictions)
